# 連続型確率変数
- 連続型確率変数はとりうる値が連続的になる。
- 計算過程で積分が含まれるなど離散型に比べ少し数式がややこしくなるが離散型と連続型に本質的な違いはない

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display

%precision 3

'%.3f'

In [2]:
# 積分を SciPy の integral module を使って実装する
from scipy import integrate

# 結果に問題はないものの warning が出る処理が含まれるため抑制する
import warnings

# 積分に関する warning を出力しないようにする
warnings.filterwarnings('ignore', category=integrate.IntegrationWarning)